In this notebook, we clean the data from the survey

To run this notebook, we need:
- a csv of the registered participants called `registeredparticipants` with columns `first` and `last`
- a csv of connections with a column called `name` and a column called `connections`

In this notebook, we
- identify each person by their id line in `registeredparticipants.csv`
- identify people who do not have unique names so we can look out for them in the code
- figure out all the ways people typed names.  Many people used first names, first and last names, or first name and last initial.  There were also many nicknames used and misspellings
- create a new dataframe called with two columns: name and connection.  Each row corresponds to a connection made at USTARS. Entries in the dataframe are integers which are the corresponding id number from `registeredparticipants.csv`.
- delete repeated connections from the dataframe df

We save the csv as `connections-clean.csv`

## Importing data and packages

In [1]:
import pandas as pd

In [2]:
all_part = pd.read_csv('registeredparticipants.csv', encoding='latin1')

In [28]:
connect = pd.read_csv('connections.csv', encoding ='latin1')

## Adjustments to the Registration List

In [29]:
#Make a full name column
full = []
for i in range(len(all_part)):
    full.append(all_part.iloc[i]['first'].strip() + ' ' + all_part.iloc[i]['last'].strip())
    
all_part['full']=full

In [30]:
#make a first name last initial column
initial = []
for i in range(len(all_part)):
    initial.append(all_part.iloc[i]['first'].strip() + ' ' + all_part.iloc[i]['last'].strip()[0])
    
all_part['initial']=initial

We look for matching names in the registration list

In [32]:
repeats=[]
for i in range(len(all_part['first'].value_counts())):
    if all_part['first'].value_counts()[i] >1:
        print(all_part['first'].value_counts().index[i])
        repeats.append(all_part['first'].value_counts().index[i])

Paige
Eric


But one Paige didn't come, so we'll remove her from the list

In [33]:
repeats.remove('Paige')

We make sure that we understand how people typed their own names

In [34]:
for i in range(len(connect)):
    person= connect.iloc[i]['name']
    person=person.strip()
    if person in repeats:
        print('Repeat: '+person)
    if person not in all_part['first'].values:
        if person not in all_part['full'].values:
            print("'"+person+ "'")

'Sofía'
'Antwon Park'
'Sofía'
'Sofía'
'Sofía'
'Valerie Batino'
'Connor N.'
'Sofía'
'Sofía'
'Sofía'
'Sofía'
'Sofia'
'Sofía'
'Sofía'
'Sofía'
'Antwon Park'
'Sofía'
'Sofía'
'Sofía'
'Sofía'
'Sofía'
'Sofía'
'Elizabeth crow'
'Christopher O'Neill'


## Establishing aliases

In [42]:
#Create a dictionary for the various ways people are called
#Keys are what they are called, and values are ID numbers

#This section has been redacted on the public version, 
#because I don't believe people need to know how their names were misspelled

## Creating the new dataframe

In [43]:
def get_id(name):
    '''a helper function that inputs a string and outputs the id number
    If it can't find it, prints the name and quits'''
    name=name.strip()
    #print(name)
    if name in repeats:
        #raise ValueError('Repeated name')
        return 80 #because only one Eric seemed active
    if name == 'Paige':
        return 33 #because the other page was not present
    elif name in all_part['first'].values:
        return all_part.loc[all_part['first']==name].index[0]
    elif name in all_part['full'].values:
        return all_part.loc[all_part['full']==name].index[0]
    elif name.strip('.') in all_part['initial'].values:
        #print('NAME INITIAL ' + name)
        return all_part.loc[all_part['initial']==name.strip('.')].index[0]
    elif name in nicknames.keys():
        return nicknames[name]
    
    else:
        raise ValueError('Cannot find name in get_id: ' + name)
        

In [44]:
df=pd.DataFrame(columns=['name','connection'])

for i in range(len(connect)):
    #entries separated by commas
    connections=connect.iloc[i]['connections']
    connections=connections.strip()
    while ',' in connections: 
        #read name until comma
        newname=''
        while connections[0] != ',':
            newname=newname+connections[0]
            connections=connections[1:]
            #print(connections)
        newname=newname.strip()
        
        #add to list
        if newname in all_part['first'].values:
            #print('here')
            name= get_id(connect.iloc[i]['name'])
            connection=get_id(newname)
            df.loc[len(df)]=[name, connection]
            
        elif newname in all_part['full'].values:
            name= get_id(connect.iloc[i]['name'])
            connection=get_id(newname)
            df.loc[len(df)]=[name, connection]
            
        elif newname in nicknames.keys():
            name= get_id(connect.iloc[i]['name'])
            connection=get_id(newname)
            df.loc[len(df)]=[name, connection]
            
        elif newname.strip('.') in all_part['initial'].values:
            name= get_id(connect.iloc[i]['name'])
            connection=get_id(newname)
            df.loc[len(df)]=[name, connection]
            
        else:
            break 
            #print('HERE1 ' + newname)
            #There is a single value Tessa that I can't figure out who it is!
            
        
        connections=connections[1:]
        
        #single entries
        
    if (',' not in connections) and connections !='':
        #print('test ' + connections)
        #connection= connect.iloc[i]['connections']
        connections=connections.strip()
        if connections in all_part['first'].values:
            #print('here')
            name= get_id(connect.iloc[i]['name'])
            connection=get_id(connections)
            df.loc[len(df)]=[name, connection]

        elif connections in all_part['full'].values:
            name= get_id(connect.iloc[i]['name'])
            connection=get_id(connections)
            df.loc[len(df)]=[name, connection]
            
        elif connections.strip('.') in all_part['initial'].values:
            name= get_id(connect.iloc[i]['name'])
            connection=get_id(connections)
            df.loc[len(df)]=[name, connection]

        elif connections in nicknames.keys():
            name= get_id(connect.iloc[i]['name'])
            connection=get_id(connections)
            df.loc[len(df)]=[name, connection]

        else:
            print('HERE2 ' +connections) 



In [45]:
len(df)

348

In [46]:
df.head()

,name,connection
0,76,82
1,76,70
2,76,7
3,76,0
4,76,69


Finally, we create a database of unique connections (key is smaller id) and export as a csv

In [47]:
finaldf=pd.DataFrame(columns=['name','conn'])
for i in range(len(df)):
    if df.iloc[i]['name'] != df.iloc[i]['connection']:
        if df.iloc[i]['name'] < df.iloc[i]['connection']:
            person1=df.iloc[i]['name']
            person2=df.iloc[i]['connection']
        else:
            person2=df.iloc[i]['name']
            person1=df.iloc[i]['connection']
        
        isthere=False
        for j in range(len(finaldf)):
            if finaldf.iloc[j]['name']==person1:
                if finaldf.iloc[j]['conn']==person2:
                    isthere=True
                    break
        
        if isthere==False:
#             print('here')
#             print(person1)
            finaldf.loc[len(finaldf)]=[person1, person2]
            
        
    

In [48]:
finaldf.head()

,name,conn
0,76,82
1,70,76
2,7,76
3,0,76
4,69,76


In [49]:
len(finaldf)

264

In [50]:
finaldf.to_csv('clean_connections.csv', index=False)